Main Caption Function

In [1]:
import subprocess
import os

# Specify the folder containing the WAV files
wav_folder = 'D:/AfterEffectsBot/Assets/audios'  # Replace with your actual path

# List all files in the folder
wav_files = [file for file in os.listdir(wav_folder) if file.startswith('youtube_audio') and file.endswith('.wav')]

# Loop through each WAV file in the folder
for wav_file in wav_files:
    command = f"whisperx {os.path.join(wav_folder, wav_file)} --model medium.en --align_model WAV2VEC2_ASR_LARGE_LV60K_960H"
    
    # Execute the command
    subprocess.run(command, shell=True)


JSON To Word TO word Srt

In [2]:
import json
import os

def seconds_to_srt_time(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = int((seconds - int(seconds)) * 1000)
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d},{milliseconds:03d}"

# Specify the folder containing the JSON files
json_folder = 'D:\Youtube Bot Audio'  # Replace with your actual path

# Specify the destination folder for the SRT files
output_folder = 'D:\Youtube Bot Audio'  # Replace with your desired output path

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all JSON files in the folder
json_files = [file for file in os.listdir(json_folder) if file.startswith('youtube_audio') and file.endswith('.json')]

for json_file in json_files:
    input_json_file = os.path.join(json_folder, json_file)
    output_file_name = f'caption_of_{json_file[:-5]}.srt'
    output_file_path = os.path.join(output_folder, output_file_name)

    with open(input_json_file, "r", encoding="utf-8") as json_file:
        data = json.load(json_file)

    segments = data["segments"]

    with open(output_file_path, "w", encoding="utf-8") as srt_file:
        word_number = 1
        for segment in segments:
            start_time = segment["start"]
            for word in segment["words"]:
                # Check if the "end" key exists in the word dictionary
                if "end" in word:
                    end_time = word["end"]
                else:
                    # If "end" key is missing, assume a small duration (e.g., 0.1 seconds)
                    end_time = start_time + 0.1
                text = word["word"]
                srt_file.write(f"{word_number}\n")
                srt_file.write(f"{seconds_to_srt_time(start_time)} --> {seconds_to_srt_time(end_time)}\n")
                srt_file.write(f"{text}\n\n")
                word_number += 1
                start_time = end_time  # Update start time for the next word

    print(f"SRT file '{output_file_name}' with corrected timestamps has been generated successfully in the '{output_folder}' folder.")


SRT file 'caption_of_youtube_audio1.srt' with corrected timestamps has been generated successfully in the 'D:\Youtube Bot Audio' folder.
SRT file 'caption_of_youtube_audio2.srt' with corrected timestamps has been generated successfully in the 'D:\Youtube Bot Audio' folder.
SRT file 'caption_of_youtube_audio3.srt' with corrected timestamps has been generated successfully in the 'D:\Youtube Bot Audio' folder.
SRT file 'caption_of_youtube_audio4.srt' with corrected timestamps has been generated successfully in the 'D:\Youtube Bot Audio' folder.


In [3]:
import os
import glob

def combine_words_to_caption(input_srt_file, output_srt_file):
    with open(input_srt_file, 'r', encoding='utf-8') as file:
        srt_content = file.read().splitlines()

    combined_srt = []
    words = []
    total_word_count = 0
    timestamps = []
    caption_index = 1

    def add_caption():
        nonlocal combined_srt, caption_index, timestamps, words
        if timestamps and len(timestamps) > 0:
            combined_srt.append(str(caption_index))
            if len(timestamps) == 1:
                combined_srt.append(timestamps[0])
            else:
                start_time, end_time = timestamps[0].split(" --> ")
                combined_srt.append(f"{start_time} --> {end_time}")
            combined_srt.append(" ".join(words))
            combined_srt.append("")
            caption_index += 1
        timestamps = []

    for line in srt_content:
        line = line.strip()
        if line.isdigit():
            add_caption()
        elif "-->" in line:
            timestamps.append(line)
        elif line:
            words.extend(line.split())
            total_word_count += len(line.split())

            while total_word_count >= 25:
                add_caption()
                words = words[25:]
                total_word_count = len(words)

    add_caption()

    with open(output_srt_file, 'w', encoding='utf-8') as file:
        file.write("\n".join(combined_srt))

if __name__ == '__main__':
    input_files = glob.glob('caption_of_youtube_audio*.srt')
    for input_srt_file in input_files:
        output_srt_file = f'output_combined_with_timestamps_{os.path.basename(input_srt_file)}'
        combine_words_to_caption(input_srt_file, output_srt_file)


In [4]:
import os
import glob

def modify_srt_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as input_srt:
        lines = input_srt.readlines()

    modified_lines = []
    captions = []

    for line in lines:
        if line.strip():
            captions.append(line.strip())
        else:
            if captions:
                modified_lines.append(captions)
            captions = []

    for i in range(len(modified_lines) - 1):
        captions = modified_lines[i]
        next_captions = modified_lines[i + 1]

        if len(captions) == 3 and len(next_captions) == 3:
            start_time = captions[1].split(" --> ")[0]
            end_time = next_captions[1].split(" --> ")[0]

            modified_lines[i][1] = f"{start_time} --> {end_time}"

    with open(output_file, 'w', encoding='utf-8') as output_srt:
        for captions in modified_lines:
            output_srt.write('\n'.join(captions) + '\n\n')  # Add an extra newline to separate captions

if __name__ == "__main__":
    input_files = glob.glob('output_combined_with_timestamps_caption_of_youtube_audio*.srt')
    for input_srt_file in input_files:
        output_srt_file = f'modified_{os.path.basename(input_srt_file)}'
        modify_srt_file(input_srt_file, output_srt_file)
        print(f"Modified SRT file saved to {output_srt_file}")


Modified SRT file saved to modified_output_combined_with_timestamps_caption_of_youtube_audio1.srt
Modified SRT file saved to modified_output_combined_with_timestamps_caption_of_youtube_audio2.srt
Modified SRT file saved to modified_output_combined_with_timestamps_caption_of_youtube_audio3.srt
Modified SRT file saved to modified_output_combined_with_timestamps_caption_of_youtube_audio4.srt


In [5]:
import os
import glob

def insert_line_breaks(input_srt_file, output_folder, output_index):
    with open(input_srt_file, 'r', encoding='utf-8') as file:
        srt_content = file.read()

    modified_srt = ""
    captions = srt_content.strip().split('\n\n')  # Split captions
    for caption in captions:
        lines = caption.split('\n')
        if len(lines) >= 3:
            caption_number, time_info, text = lines[0], lines[1], lines[2:]
            words = text[0].split()  # Split the text into words
            first_line = ' '.join(words[:8])  # First line with 8 words
            if len(words) > 8:
                second_line = ' '.join(words[8:17])  # Second line with 9 words if available
                third_line = ' '.join(words[17:])  # Third line with the remaining words
            else:
                second_line = ' '.join(words[8:])  # Use the remaining words for the second line
                third_line = ''  # No third line
            new_lines = [caption_number, time_info, first_line, second_line, third_line] + text[1:]  # Retain other lines
            modified_srt += '\n'.join(new_lines) + '\n\n'
        else:
            modified_srt += caption + '\n\n'  # Retain captions with less than 3 lines

    output_file = f'caption_of_youtube_audio{output_index}.srt'
    output_srt_file = os.path.join(output_folder, output_file)

    with open(output_srt_file, 'w', encoding='utf-8') as file:
        file.write(modified_srt)

if __name__ == '__main__':
    input_files = glob.glob('modified_output_combined_with_timestamps_caption_of_youtube_audio*.srt')
    output_folder = r'D:\AfterEffectsBot\Assets\\captions'
    output_index = 1
    for input_srt_file in input_files:
        insert_line_breaks(input_srt_file, output_folder, output_index)
        output_index += 1


In [6]:
import os

# Specify the folder containing the files
folder = 'D:\Youtube Bot Audio'  # Replace with your actual path

# List all SRT files in the folder
srt_files = [file for file in os.listdir(folder) if file.lower().endswith('.srt')]

# Delete SRT files
for srt_file in srt_files:
    srt_file_path = os.path.join(folder, srt_file)
    os.remove(srt_file_path)
    print(f"SRT file '{srt_file}' has been deleted.")

# List all files starting with "youtube_audio" in the folder
youtube_audio_files = [file for file in os.listdir(folder) if file.startswith('youtube_audio')]

# Delete files starting with "youtube_audio"
for youtube_audio_file in youtube_audio_files:
    youtube_audio_file_path = os.path.join(folder, youtube_audio_file)
    os.remove(youtube_audio_file_path)
    print(f"File '{youtube_audio_file}' has been deleted.")


SRT file 'caption_of_youtube_audio1.srt' has been deleted.
SRT file 'caption_of_youtube_audio2.srt' has been deleted.
SRT file 'caption_of_youtube_audio3.srt' has been deleted.
SRT file 'caption_of_youtube_audio4.srt' has been deleted.
SRT file 'modified_output_combined_with_timestamps_caption_of_youtube_audio1.srt' has been deleted.
SRT file 'modified_output_combined_with_timestamps_caption_of_youtube_audio2.srt' has been deleted.
SRT file 'modified_output_combined_with_timestamps_caption_of_youtube_audio3.srt' has been deleted.
SRT file 'modified_output_combined_with_timestamps_caption_of_youtube_audio4.srt' has been deleted.
SRT file 'output_combined_with_timestamps_caption_of_youtube_audio1.srt' has been deleted.
SRT file 'output_combined_with_timestamps_caption_of_youtube_audio2.srt' has been deleted.
SRT file 'output_combined_with_timestamps_caption_of_youtube_audio3.srt' has been deleted.
SRT file 'output_combined_with_timestamps_caption_of_youtube_audio4.srt' has been deleted.


In [7]:
import subprocess

# Specify the path to the text file
file_path = "most_viewed_articles.txt"  # Replace with the actual path

try:
    subprocess.Popen(["notepad.exe", file_path])
except FileNotFoundError:
    print(f"File not found at '{file_path}'.")
except Exception as e:
    print(f"An error occurred: {e}")


: 